# Tugas Praktikum — SVM pada voice.csv

Pada tugas ini, kita akan membangun beberapa model SVM untuk dataset suara (`../data/voice.csv`) dengan ketentuan:
- Split data: 70:30 dan 80:20
- Kernel: linear, polynomial, dan RBF
- Tampilkan performansi akurasi tiap kombinasi split×kernel dalam bentuk tabel.

Catatan: Pastikan file `voice.csv` berada pada path relatif `../data/voice.csv` terhadap notebook ini.

1. Buatlah model SVM dengan menggunakan data voice.csv dengan ketentuan,
    - Split data dengan rasio 70:30 dan 80:20 untuk setiap model yang akan dibangun.
        - Gunakan model dengan kernel linier.
        - Gunakan model dengan kernel polynomial.
        - Gunakan model dengan kernel RBF.

    - Tabulasikan performansi setiap split dan kernel berdasarkan metrik akurasi.
2. Gunakan data pada praktikum 5 untuk membuat model klasifikasi siang dan malam menggunakan SVM dengan kernel RBF menggunakan fitur histrogram Gunakan rasio 80:20. Anda dapat bereksperimen dengan hyperparameter tunning dari kernel RBF. Catat performansi akurasinya!

In [ ]:
# SVM pada dataset voice.csv
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 1) Load dataset
csv_path = Path("../data/voice.csv")
if not csv_path.exists():
    raise FileNotFoundError(f"Dataset tidak ditemukan di {csv_path.resolve()}. Pastikan file voice.csv berada pada path tersebut.")

voice = pd.read_csv(csv_path)

# 2) Basic preprocessing
# - Pastikan kolom label bernama 'label' (male/female)
# - Konversi ke 0/1 (female=1, male=0) agar konsisten seperti tugas sebelumnya
if 'label' not in voice.columns:
    raise KeyError("Kolom 'label' tidak ditemukan pada dataset voice.csv")

# Coerce numeric for all feature columns except label
feature_cols = [c for c in voice.columns if c != 'label']
voice[feature_cols] = voice[feature_cols].apply(pd.to_numeric, errors='coerce')
voice.dropna(inplace=True)

label_map = {'female': 1, 'male': 0}
voice['label'] = voice['label'].map(label_map)

if voice['label'].isna().any():
    raise ValueError("Ditemukan label di luar {female,male}. Harap periksa dataset.")

X = voice[feature_cols].values
y = voice['label'].values.astype(int)

# 3) Define experiments
splits = [0.3, 0.2]  # test_size untuk 70:30 dan 80:20
kernels = [
    {"name": "linear", "svc_kwargs": {"kernel": "linear", "random_state": 42}},
    {"name": "poly",   "svc_kwargs": {"kernel": "poly",   "degree": 3, "random_state": 42}},
    {"name": "rbf",    "svc_kwargs": {"kernel": "rbf",                "random_state": 42}},
]

# 4) Run experiments and collect results
results = []
for test_size in splits:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )

    for k in kernels:
        # Pipeline: StandardScaler -> SVC
        pipe = make_pipeline(
            StandardScaler(),
            SVC(**k["svc_kwargs"])  # default C=1.0, gamma='scale' untuk rbf/poly
        )
        pipe.fit(X_train, y_train)

        # Accuracy on train and test
        acc_train = accuracy_score(y_train, pipe.predict(X_train))
        acc_test = accuracy_score(y_test, pipe.predict(X_test))

        results.append({
            "split": f"{int((1-test_size)*100)}:{int(test_size*100)}",
            "kernel": k["name"],
            "acc_train": acc_train,
            "acc_test": acc_test,
        })

results_df = pd.DataFrame(results).sort_values(by=["split", "kernel"]).reset_index(drop=True)
print("Ringkasan akurasi per kombinasi split×kernel:")
print(results_df)

# Optional: tampilkan tabel pivot akurasi test untuk perbandingan cepat
try:
    display(results_df.pivot(index="kernel", columns="split", values="acc_test"))
except Exception:
    pass

Ringkasan akurasi per kombinasi split×kernel:
   split  kernel  acc_train  acc_test
0  70:30  linear   0.977898  0.969506
1  70:30    poly   0.967975  0.949527
2  70:30     rbf   0.984664  0.981073
3  80:20  linear   0.977901  0.968454
4  80:20    poly   0.968035  0.952681
5  80:20     rbf   0.986188  0.979495


split,70:30,80:20
kernel,,
linear,0.969506,0.968454
poly,0.949527,0.952681
rbf,0.981073,0.979495
